In [25]:
from __future__ import division # For python 2.*

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn import *
from sklearn.metrics import roc_curve, auc

np.random.seed(0)
%matplotlib inline

X = np.genfromtxt('uci-cs178-f21/X_train.txt', delimiter=',')
Y = np.genfromtxt('uci-cs178-f21/Y_train.txt', delimiter=',')

X,Y = ml.shuffleData(X,Y)
# Fetureset1_X = X

# X = X[:,0:41]
Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.80)

Xte = np.genfromtxt('uci-cs178-f21/X_test.txt', delimiter=',')
# Xte = Xte[:,:41]

Xtr, param = ml.transforms.rescale(Xtr)
Xva, _ = ml.transforms.rescale(Xva,param)
Xte, _ = ml.transforms.rescale(Xte,param)

In [26]:
print(Xte.shape)
print(Xtr.shape)
print(Xva.shape)

(7422, 107)
(5938, 107)
(1485, 107)


In [16]:
# print(X.shape)
# print(Y.shape)
# print(Xte.shape)

# Fetureset1_X = X[:,0:41]
# print(Fetureset1_X.shape)
# Fetureset2_X = X[:,41:69]
# print(Fetureset2_X.shape)
# Fetureset3_X = X[:,69:107]
# print(Fetureset3_X.shape)

# print(Fetureset1_X[0])
# print(Fetureset2_X[0])
# print(Fetureset3_X[0])

# # for i in X[:1]:
# #     for j in i :
# #         x += 1
# #         print(x,j)


In [33]:
nBag = 100
classifiers2 = [ None ] * nBag

mt,nt = Xte.shape

Ye = np.zeros((mt,2))
auc_list = []

for i in range(nBag):
    Xi,Yi = ml.bootstrapData(Xtr,Ytr,mt) 
    classifiers2[i] = ml.dtree.treeClassify(Xi, Yi, maxDepth = 25 , minLeaf = 4, minParent = 512, nFeatures = 41)
    #Train a model on data Xi,Yi
    auc_list.append(classifiers2[i].auc(Xva,Yva))
    #Train a model on data Xi,Yi
    Ye += classifiers2[i].predictSoft(Xte)
learner = Ye / 25
print(np.mean(auc_list))

0.6891955487751786


In [32]:
print(learner)

[[2.52608891 1.47391109]
 [3.14210265 0.85789735]
 [1.85569989 2.14430011]
 ...
 [2.07454872 1.92545128]
 [0.33362584 3.66637416]
 [2.04357096 1.95642904]]


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn import gaussian_process
from sklearn import linear_model


In [13]:
rf = RandomForestClassifier(n_estimators= 100, min_samples_split = 512,max_features = 41,min_samples_leaf = 4, bootstrap=True, max_depth=25)
rf.fit(Xtr, Ytr)

pred_soft = rf.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
roc_auc_tr = auc(false_positive_rate, true_positive_rate)

pred_soft = rf.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
roc_auc_va = auc(false_positive_rate, true_positive_rate)


print ("max_features = " + str(41))
print ("Training: " + str(roc_auc_tr))
print ("Validation: " + str(roc_auc_va))


max_features = 41
Training: 0.7811161581178616
Validation: 0.7422487229349277


In [34]:
pred_soft[:,1]

array([0.4795793 , 0.48953922, 0.50037116, ..., 0.27748404, 0.52129013,
       0.68978669])

In [ ]:
max = 0
currk = 0

for mf in range(10,40):
    rf = RandomForestClassifier(n_estimators= 50, max_features=mf, bootstrap=True, max_depth=10)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = mf
    
    print ("maxF = " + str(mf))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print (max, currk)

In [ ]:
max = 0
currk = 0

for mf in range(10,40):
    rf = RandomForestClassifier(n_estimators=100, max_features=mf, bootstrap=True, max_depth=10)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = mf
    
#     print ("maxF = " + str(mf))
#     print ("Training: " + str(roc_auc_tr))
#     print ("Validation: " + str(roc_auc_va))
    
print (max, mf)

In [ ]:
max = 0
currk = 0
depths=[10,20,30,40,50,60,70]
for d in depths:
    rf = RandomForestClassifier(n_estimators=100, max_features=39, bootstrap=True, max_depth=d)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = d
    
    print ("depth = " + str(d))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print (max, currk)

In [ ]:
max = 0
currk = 0
nbag=[(50,28),(100,39)]
for b,mf in nbag:
    rf = RandomForestClassifier(n_estimators= 50, max_features= 39, bootstrap=True, max_depth= 10)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = d
    
    print ("nbag = " + str(b))
    print ("mf = " + str(mf))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print (max, currk)